In [1]:
from pymongo import MongoClient
from bigchaindb_driver import BigchainDB
import time

In [2]:
MDB = MongoClient('localhost', 27017)
BDB = BigchainDB('http://localhost:9984')

In [3]:
MDB.list_database_names()

['admin', 'bigchain', 'config', 'local']

In [4]:
mbc = MDB.bigchain

In [5]:
mbc.list_collection_names()

['transactions',
 'validators',
 'elections',
 'pre_commit',
 'metadata',
 'abci_chains',
 'blocks',
 'assets',
 'utxos']

In [6]:
for a in mbc.assets.find({'data.asset_type':'university'}):
    print(a)

{'_id': ObjectId('5bf0099802943d0001a738b8'), 'data': {'asset_type': 'university', 'name': 'University of Cape Town', 'physical_address': 'Rondebosch, Cape Town, 7700', 'short': 'UCT'}, 'id': 'a50bb3c754dde6013a4369c1ad6bc7bee325a2591d81a135f9ba7cb3ac279c51'}


In [7]:
for m in mbc.metadata.find({'id': 'a50bb3c754dde6013a4369c1ad6bc7bee325a2591d81a135f9ba7cb3ac279c51'}):
    print(m)

{'_id': ObjectId('5bf0099802943d0001a738b7'), 'id': 'a50bb3c754dde6013a4369c1ad6bc7bee325a2591d81a135f9ba7cb3ac279c51', 'metadata': {'timestamp': '2018-07-20'}}


In [8]:
for a in mbc.assets.find({'data.asset_type':'mark', 'data.student_address': '4'}):
    print(a)
    for m in mbc.metadata.find({'id': a['id']}):
        print(m)

{'_id': ObjectId('5bf009bb02943d0001a73b37'), 'data': {'asset_type': 'mark', 'course_id': '7b39a57a1479948460b78b5bab1036db1d7d518c40bb697e1b2c716cc34a5a82', 'degree_id': '8068da528a0a53b44006ae4de900fa34830e0a782a8c3693b02351b331cd0118', 'student_address': '4', 'type': 'Class Participation'}, 'id': 'e436b248398fa7ac9d62c021fcfcd238b6dac6fc7087ad9679b2863bc867c578'}
{'_id': ObjectId('5bf009bb02943d0001a73b36'), 'id': 'e436b248398fa7ac9d62c021fcfcd238b6dac6fc7087ad9679b2863bc867c578', 'metadata': {'mark': 50, 'timestamp': '2018-08-03'}}
{'_id': ObjectId('5bf009bc02943d0001a73b3b'), 'data': {'asset_type': 'mark', 'course_id': '7b39a57a1479948460b78b5bab1036db1d7d518c40bb697e1b2c716cc34a5a82', 'degree_id': '8068da528a0a53b44006ae4de900fa34830e0a782a8c3693b02351b331cd0118', 'student_address': '4', 'type': 'Tutorial Participation'}, 'id': 'b6d7dc94258dde625fed71aba723d9c4f082875e19eeef170bcbd0d39b82e13d'}
{'_id': ObjectId('5bf009bc02943d0001a73b3a'), 'id': 'b6d7dc94258dde625fed71aba723d9c4f

In [13]:
def _get_marks_by_student(address):
    files = BDB.assets.get(search=address)
    (files, course_data) = _retrieve_course_data(files, address)
    degree_data = _retrieve_degree_data(files)
    mark_data = _retrieve_mark_data(files, course_data)
    return {'degree_data': degree_data, 'mark_data': mark_data}

def _retrieve_degree_data(files):
    if files:
        degree_id = files[0].get('data').get('degree_id')
        degree = BDB.transactions.get(asset_id=degree_id)
        degree_data = {
            'name': degree[0].get('asset').get('data').get('name'),
            'level': degree[0].get('asset').get('data').get('level'),
            'courses': degree[-1].get('metadata').get('courses')
        }
        return degree_data

def _retrieve_course_data(files, address):
    (files, course_ids) = _retrieve_course_ids(files, address)
    course_data = _retrieve_course_information(course_ids)
    return (files, course_data)

def _retrieve_course_ids(files, address):
    courses = []
    for f in files[:]:
        if (f.get('data').get('asset_type') == 'mark') and (f.get('data').get('student_address') == address):
            courses.append(f.get('data').get('course_id'))
        else:
            files.remove(f)
    return (files, list(set(courses)))

def _retrieve_course_information(course_ids):
    course_data = dict()
    for course_id in course_ids:
        course = BDB.transactions.get(asset_id=course_id)
        course_data[course_id] = {
            'name': course[0].get('asset').get('data').get('name'), 
            'lecturer': course[0].get('asset').get('data').get('lecturer'),
            'components': course[-1].get('metadata').get('components')
        }
    return course_data

def _retrieve_mark_data(mark_files, course_data):
    mark_data = dict()
    for f in mark_files:
        course_id = f.get('data').get('course_id')
        mark_type = f.get('data').get('type')
        mark_file = BDB.transactions.get(asset_id=f.get('id'))[-1]
        mark = mark_file.get('metadata').get('mark')
        timestamp = mark_file.get('metadata').get('timestamp')
        weighting = (item for item in course_data.get(course_id).get('components') if item["type"] == mark_type).__next__().get('weighting')
        if mark_data.get(course_id):
            mark_data.get(course_id).get('components')[mark_type] = {
                'mark': mark, 
                'weighting': weighting, 
                'timestamp': timestamp
            }
            if mark_data.get(course_id).get('year') < timestamp[:4]:
                mark_data.get(course_id)['year'] = timestamp[:4]
        else:
            mark_data[course_id] = {
                'name': course_data.get(course_id).get('name'), 
                'lecturer': course_data.get(course_id).get('lecturer'), 
                'year': timestamp[:4],
                'components': {
                    mark_type: {
                        'mark': mark, 
                        'weighting': weighting, 
                        'timestamp': timestamp
                    }
                }
            }
    return mark_data

def _get_assets_by_key(asset, key, value, meta_flag):
    files = BDB.assets.get(search=value)
    assets = []
    for f in files:
        if (f.get('data').get('asset_type') == asset) and (f.get('data').get(key) == value):
            if meta_flag:
                asset_id = f.get('id')
                metadata = BDB.transactions.get(asset_id=asset_id)[-1].get('metadata')
                assets.append({**f, **{'metadata': metadata}})
            else: 
                assets.append(f)
    return assets

def _get_course_marks_by_lecturer(lecturer):
    courses = _get_assets_by_key('course', 'lecturer', lecturer, True)
    course_ids = [item.get('id') for item in courses]
    marks_per_course = dict()
    student_addresses = set()
    for i, course_id in enumerate(course_ids):
        marks = _get_assets_by_key('mark', 'course_id', course_id, True)
        course_marks = dict()
        for mark in marks:
            student_address = mark.get('data').get('student_address')
            mark_data = {'id': mark.get('id'), 'type': mark.get('data').get('type'), 'mark': mark.get('metadata').get('mark')}
            if not course_marks.get(student_address):
                course_marks[student_address] = [mark_data]
            else:
                course_marks[student_address].append(mark_data)
            student_addresses.add(student_address)
        marks_per_course[course_id] = {'name': courses[i].get('data').get('name'), 'components': courses[i].get('metadata').get('components'), 'course_marks': course_marks}
    return {'student_addresses': list(student_addresses), 'marks_per_course': marks_per_course}

In [14]:
_get_course_marks_by_lecturer('Allan Davids')

{'student_addresses': ['8',
  '3',
  '5',
  '7',
  '10',
  '11',
  '4',
  '12',
  '9',
  '6',
  '1',
  '2',
  '0',
  '13'],
 'marks_per_course': {'fb3ba7d4779407e85c707c9279f3d1fc3a899596a45335800a92c34f6f5e52e7': {'name': 'Data Visualization',
   'components': [{'required': True, 'type': 'Test', 'weighting': 0.15},
    {'required': True, 'type': 'Project', 'weighting': 0.45},
    {'required': True, 'type': 'Tutorials', 'weighting': 0.1},
    {'required': True, 'type': 'Exam', 'weighting': 0.3}],
   'course_marks': {'0': [{'id': '2c1a183d553058ad93930c857cf9c8e1b322ecedb0f7b60cfeed124b71bd2fbd',
      'type': 'Test',
      'mark': 65},
     {'id': 'cecb612d408caa4baf01cf7b874b5fcb035740b0d99639be146a6e3987b6620f',
      'type': 'Project',
      'mark': 74},
     {'id': '279eb50574631a9fb7c98c757ad2c6752dbc769e74d6ac69bad7a482d79d56e0',
      'type': 'Tutorials',
      'mark': 88},
     {'id': 'f90482480ed3f71ce91838e2e6014fb2da42adb8021bca32c64a805df219c2d3',
      'type': 'Exam',
    

In [ ]:
t0 = time.time()
_get_marks_by_student('4')
print(time.time() - t0)

In [ ]:
def get_asset_data(_id):
    asset = mbc.assets.find_one({'id': _id})
    return asset['data']

def get_asset_metadata(asset_id):
    transactions = list(mbc.transactions.find({'asset.id': asset_id}))
    if transactions:
        return mbc.metadata.find_one({'id': transactions[-1]['id']})['metadata']
    else:
        return mbc.metadata.find_one({'id': asset_id})['metadata']

In [ ]:
def get_student_mark_assets(student_address):
    return list(mbc.assets.find({'data.asset_type':'mark', 'data.student_address': student_address}))

def process(mark_assets):
    course_ids = list()
    degree_ids = list()
    marks = dict()

    for m in mark_assets:
        course_id = m['data']['course_id']
        degree_id = m['data']['degree_id']
        mark_metadata = get_asset_metadata(m['id'])
        
        if not marks.get(course_id):
            marks[course_id] = dict()
            marks[course_id]['components'] = {m['data']['type']: {'mark': mark_metadata['mark'], 'timestamp': mark_metadata['timestamp']}, 'degree_id': m['data']['degree_id']}
        else:
            marks[course_id]['components'][m['data']['type']] = {'mark': mark_metadata['mark'], 'timestamp': mark_metadata['timestamp'], 'degree_id': m['data']['degree_id']}
            
        course_ids.append(course_id)
        degree_ids.append(degree_id)
        
    course_ids = list(set(course_ids))
    degree_ids = list(set(degree_ids))
    
    return (degree_ids, course_ids, marks)

def add_course_info(marks, course_ids):
    for course_id in course_ids:
        course_asset = get_asset_data(course_id)
        course_metadata = get_asset_metadata(course_id)
        
        if marks.get(course_id).get('year', '1900') < course_metadata['timestamp'][:4]:
            marks.get(course_id)['year'] = course_metadata['timestamp'][:4]
        
        marks[course_id] = {**marks[course_id], **course_asset}
        for c in course_metadata['components']:
            marks[course_id]['components'][c['type']]['weighting'] = c['weighting']
            
    return marks

def add_degree_info(marks, degree_ids):
    degree_data = dict()
    for degree_id in degree_ids:
        degree_data[degree_id] = {**get_asset_data(degree_id), **get_asset_metadata(degree_id)}
    return {'degree_data': degree_data, 'mark_data': marks}

def _get_marks_by_student2(student_address):
    mark_assets = get_student_mark_assets(student_address)
    degree_ids, course_ids, mark_data = process(mark_assets)
    mark_data = add_course_info(mark_data, course_ids)
    mark_data = add_degree_info(mark_data, degree_ids)
    return mark_data

In [ ]:
t0 = time.time()
_get_marks_by_student2('4')
print(time.time() - t0)